In [5]:
!pip install joblib

You should consider upgrading via the '/Users/wei-yi/anaconda3/bin/python -m pip install --upgrade pip' command.


In [7]:
# import packages
import pandas as pd
# import seaborn as sb
import matplotlib.pyplot as plt
import numpy as np
import datetime
import re
import joblib 
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from scipy.interpolate import interpn
from collections import defaultdict

In [35]:
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [36]:
sales_means =pd.read_csv('./data/sales_means.csv', sep='|').drop(columns=["predicted_promotion"])
scaler = joblib.load('scaler.pkl') 
df_test = pd.read_csv('data/orders0206_test.csv', sep='|', parse_dates=['time'])
df_test['date'] = [d.date() for d in df_test['time']]
df_train = pd.read_csv('data/orders0206_train.csv', sep='|', parse_dates=['time'])
df_train['date'] = [d.date() for d in df_train['time']]
bundles = pd.read_csv('./data/bundles.csv', sep=',')

In [50]:
sales_means.head()

,count,weekGroup,itemID,customerRating,manufacturer,brand,category1,category3,dateFirstSell,soldWeekly,...,avg_salesPrice_lag_5,brand_avg_avg_salesPrice_lag_5,brand_sum_count_lag_5,brand_avg_count_lag_5,category1_avg_avg_salesPrice_lag_5,category1_sum_count_lag_5,category1_avg_count_lag_5,category3_avg_avg_salesPrice_lag_5,category3_sum_count_lag_5,category3_avg_count_lag_5
0,0.000414,1,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
1,0.064817,2,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
2,0.007248,3,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
3,0.000621,4,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0
4,0.000207,5,1,0.876,1,0,0,0,1,False,...,0.000296,0.022205,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0


In [58]:
bundles[bundles['itemID']==1]

,itemID,bundles,free_bundles
2021,1,True,False


In [59]:
sales_means = pd.merge(sales_means, bundles, on='itemID', how='left')

In [60]:
sales_means.head()

,count,weekGroup,itemID,customerRating,manufacturer,brand,category1,category3,dateFirstSell,soldWeekly,...,brand_sum_count_lag_5,brand_avg_count_lag_5,category1_avg_avg_salesPrice_lag_5,category1_sum_count_lag_5,category1_avg_count_lag_5,category3_avg_avg_salesPrice_lag_5,category3_sum_count_lag_5,category3_avg_count_lag_5,bundles,free_bundles
0,0.000414,1,1,0.876,1,0,0,0,1,False,...,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0,True,False
1,0.064817,2,1,0.876,1,0,0,0,1,False,...,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0,True,False
2,0.007248,3,1,0.876,1,0,0,0,1,False,...,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0,True,False
3,0.000621,4,1,0.876,1,0,0,0,1,False,...,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0,True,False
4,0.000207,5,1,0.876,1,0,0,0,1,False,...,0.0,0.0,0.290938,0.0,0.0,0.107282,0.0,0.0,True,False


In [108]:
sales_means.sample(frac=1).reset_index(drop=True)
X_train = sales_means.tail(len(sales_means)-2000)#[(sales_means['weekGroup']<=9)]# & (sales_means['weekGroup']==10)]
X_cv =  sales_means.head(2000)#[sales_means['weekGroup']==10]
X_test = sales_means[sales_means['weekGroup']==11]
Y_train = X_train['count']
Y_cv = X_cv['count']
Y_test = X_test['count']
del X_train['count']
del X_cv['count']
del X_test['count']
del X_train['weekGroup']
del X_cv['weekGroup']
del X_test['weekGroup']

In [109]:
del X_train['itemID']
del X_cv['itemID']

In [110]:
X_train.columns

Index(['customerRating', 'manufacturer', 'brand', 'category1', 'category3',
       'dateFirstSell', 'soldWeekly', 'recentlySold', 'count_lag_1',
       'avg_salesPrice_lag_1', 'brand_avg_avg_salesPrice_lag_1',
       'brand_sum_count_lag_1', 'brand_avg_count_lag_1',
       'category1_avg_avg_salesPrice_lag_1', 'category1_sum_count_lag_1',
       'category1_avg_count_lag_1', 'category3_avg_avg_salesPrice_lag_1',
       'category3_sum_count_lag_1', 'category3_avg_count_lag_1', 'count_lag_2',
       'avg_salesPrice_lag_2', 'brand_avg_avg_salesPrice_lag_2',
       'brand_sum_count_lag_2', 'brand_avg_count_lag_2',
       'category1_avg_avg_salesPrice_lag_2', 'category1_sum_count_lag_2',
       'category1_avg_count_lag_2', 'category3_avg_avg_salesPrice_lag_2',
       'category3_sum_count_lag_2', 'category3_avg_count_lag_2', 'count_lag_3',
       'avg_salesPrice_lag_3', 'brand_avg_avg_salesPrice_lag_3',
       'brand_sum_count_lag_3', 'brand_avg_count_lag_3',
       'category1_avg_avg_salesPr

In [111]:
from xgboost import XGBRegressor
model = XGBRegressor(
    booster = 'gbtree', 
    colsample_bytree = 0.8, 
    gamma = 0, 
    learning_rate = 0.1, 
    max_depth= 3, 
    min_child_weight= 1, 
    n_estimators= 300, 
    objective= 'reg:logistic', 
    seed= 20, 
    silent= 1, 
    subsample= 0.8)
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_cv, Y_cv)], 
    verbose=True, 
    early_stopping_rounds = 10)

[15:43:13] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:480: 
Parameters: { silent } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	validation_0-rmse:0.44237	validation_1-rmse:0.44773
Multiple eval metrics have been passed: 'validation_1-rmse' will be used for early stopping.

Will train until validation_1-rmse hasn't improved in 10 rounds.
[1]	validation_0-rmse:0.39888	validation_1-rmse:0.40375
[2]	validation_0-rmse:0.35994	validation_1-rmse:0.36433
[3]	validation_0-rmse:0.32503	validation_1-rmse:0.32894
[4]	validation_0-rmse:0.29367	validation_1-rmse:0.29716
[5]	validation_0-rmse:0.26548	validation_1-rmse:0.26863
[6]	validation_0-rmse:0.24011	validation_1-rmse:0.24292
[7]	validation_0-rmse:0.21730	validation_1-rmse:0.21971
[8]	validation_0-rmse:0.19676	validat

[135]	validation_0-rmse:0.02749	validation_1-rmse:0.01034
[136]	validation_0-rmse:0.02748	validation_1-rmse:0.01034
[137]	validation_0-rmse:0.02745	validation_1-rmse:0.01034
[138]	validation_0-rmse:0.02743	validation_1-rmse:0.01033
[139]	validation_0-rmse:0.02741	validation_1-rmse:0.01033
[140]	validation_0-rmse:0.02740	validation_1-rmse:0.01033
[141]	validation_0-rmse:0.02738	validation_1-rmse:0.01033
[142]	validation_0-rmse:0.02736	validation_1-rmse:0.01033
[143]	validation_0-rmse:0.02735	validation_1-rmse:0.01033
[144]	validation_0-rmse:0.02733	validation_1-rmse:0.01033
[145]	validation_0-rmse:0.02732	validation_1-rmse:0.01033
[146]	validation_0-rmse:0.02731	validation_1-rmse:0.01033
[147]	validation_0-rmse:0.02730	validation_1-rmse:0.01033
[148]	validation_0-rmse:0.02727	validation_1-rmse:0.01033
[149]	validation_0-rmse:0.02726	validation_1-rmse:0.01033
[150]	validation_0-rmse:0.02724	validation_1-rmse:0.01032
[151]	validation_0-rmse:0.02723	validation_1-rmse:0.01032
[152]	validati

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.8, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=300, n_jobs=0, num_parallel_tree=1,
             objective='reg:logistic', random_state=20, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, seed=20, silent=1, subsample=0.8,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [112]:
def evaluate_result(y: dict, y_pred: dict):
    monetary_value = 0
    y_pred = defaultdict(int, y_pred)  # return prediction of 0 for items without prediction
    
    for item in set(y_pred).difference(set(y)):
        y[item] = 0  # make sure that all items for which a demand has been predicted are contained in the actual demands
    
    for item, demand in y.items():
        predicted_demand = y_pred[item]
        price = product_prices[item]
        monetary_value += price * min(demand, predicted_demand)
        if predicted_demand > demand:
            monetary_value -= .6 * price * (predicted_demand - demand)
            
    return monetary_value

In [113]:
df_info = pd.read_csv('data/infos.csv', sep='|', index_col='itemID')
df_items = pd.read_csv('data/items.csv', sep='|', index_col='itemID')
df_items.head()

,brand,manufacturer,customerRating,category1,category2,category3,recommendedRetailPrice
itemID,,,,,,,
1,0,1,4.38,1,1,1,8.84
2,0,2,3.00,1,2,1,16.92
3,0,3,5.00,1,3,1,15.89
4,0,2,4.44,1,2,1,40.17
5,0,2,2.33,1,1,1,17.04


In [137]:
# actual demand
y = df_test.groupby(by='itemID')['order'].sum().to_dict()

# baseline 1 (average demand of previous 14 days)
y_baseline1 = df_train[df_train['time'] >= '2018-05-19'].groupby(by='itemID')['order'].sum().to_dict()

# baseline 2 (average demand of previous half year)
total_orders = df_train.groupby(by='itemID')['order'].sum().to_dict()
total_observed_days = (df_train['time'].dt.normalize().max() - df_train['time'].dt.normalize().min()).days
y_baseline2 = {item: orders / total_observed_days * 14 for item, orders in total_orders.items()}  # 14-day avg. demand

df_info = pd.read_csv('data/infos.csv', sep='|', index_col='itemID')
df_items = pd.read_csv('data/items.csv', sep='|', index_col='itemID')
product_prices = df_info['simulationPrice'].to_dict()

In [138]:
# apply to unseen data
y_xgboost_all = dict()
for prod in X_test.itemID.unique():
    train_predict = pd.DataFrame(scaler.inverse_transform(pd.DataFrame(model.predict(X_test[X_test.itemID == prod].drop(['itemID'], axis=1))))).rename(columns={0:'predicted_count'})
    y_xgboost_all[prod] = int(train_predict["predicted_count"].sum().round())


In [139]:
# perfect result
print(f'Perfect Result: {evaluate_result(y, y):.2f}')

# baseline 1
print(f'Baseline 1: {evaluate_result(y, y_baseline1):.2f}')

# baseline 2
print(f'Baseline 2: {evaluate_result(y, y_baseline2):.2f}')

# random forest
print(f'XGBoost: {evaluate_result(y, y_xgboost_all):.2f}')
#-205469.90
# add bundles -87550.78
#30430 run directly

Perfect Result: 7895975.87
Baseline 1: -3727365.60
Baseline 2: -1672504.21
XGBoost: 48230.75


In [140]:
# plot feature importance using built-in function
from xgboost import plot_importance
from matplotlib import pyplot
from matplotlib.pyplot import figure
# load data
# fit model no training data
model.fit(X_train, 
    Y_train)
fig,ax = plt.subplots(figsize=(15,15))
# plot feature importance
plot_importance(model,
                height=0.5,
                ax=ax,
                max_num_features=64)
pyplot.show()

AttributeError: 'Booster' object has no attribute 'fit'

In [ ]:
parameters = {#'nthread':[4], #when use hyperthread, xgboost may become slower
          'objective':["reg:logistic"],#,"reg:squaredlogerror", 'reg:linear',"reg:squarederror"; logistic was best!
          'learning_rate': [0.1, 0.2, 0.3], # 0.01, 0.3; 0.1 and 0.3 were best depending on model #so called `eta` value
          'max_depth': [3,4,5],#,4,5,6], 3 was best
          'min_child_weight': [1,10],#, 50, 200,300],
          'silent': [1],
          'subsample': [ 0.3,0.5], # depends on model
          'colsample_bytree': [ 0.3,0.5], #0.5,0.8
              'n_estimators': [300,500],
#               #'eta': [0.01, 0.1,0.3],
              'seed': [0,20,40],#, 10,40,75],
#               'eval_metric': ['rmsle','rmse'],
          'booster':['gbtree'],#,"dart"], # both models choose dart when optimizing, but gbtree performs better on test data
          'gamma': [0,5,10,20]} #0,100; 10 was best
model = GridSearchCV(xgb1,
                    parameters,
                    cv = 3,
                    n_jobs = 8,
                    verbose=True)

# train the model
model.fit(
    X_train, 
    Y_train, 
    eval_metric="rmse", 
    eval_set=[(X_train, Y_train), (X_cv, Y_cv)], 
    verbose=True, 
    early_stopping_rounds = 10)

print(model.best_score_)
print(model.best_params_)

In [180]:

def model(train_x,train_y,test_x,test_y):
    params = {
        'nthread': 10,
         'max_depth': 5,
#         'max_depth': 9,
        'task': 'train',
        'boosting_type': 'gbdt',
        'objective': 'regression_l1',
        'metric': 'mape', # this is abs(a-e)/max(1,a)
#         'num_leaves': 39,
        'num_leaves': 64,
        'learning_rate': 0.1,
       'feature_fraction': 0.8,
#         'feature_fraction': 0.8108472661400657,
#         'bagging_fraction': 0.9837558288375402,
       'bagging_fraction': 0.9,
        'bagging_freq': 5,
        'lambda_l1': 3.097758978478437,
        'lambda_l2': 2.9482537987198496,
#       'lambda_l1': 0.06,
#       'lambda_l2': 0.1,
        'verbose': 1,
        'min_child_weight': 6.996211413900573,
        'min_split_gain': 0.037310344962162616,
        }
    
    lgb_train = lgb.Dataset(train_x,train_y)
    lgb_valid = lgb.Dataset(test_x,test_y)
    model = lgb.train(params, lgb_train, 3000, valid_sets=[lgb_train, lgb_valid],early_stopping_rounds=50, verbose_eval=50)

    return model

#y_test, model = model(train_x,train_y,test_x,test_y,col)

In [181]:
# parameters = {
#     'max_depth': [4,6,8],
#     'num_leaves': [20,30,40],
# }

# gbm = lgb.LGBMClassifier(objective = 'binary',
#                          is_unbalance = True,
#                          metric = 'binary_logloss,auc',
#                          max_depth = 6,
#                          num_leaves = 40,
#                          learning_rate = 0.1,
#                          feature_fraction = 0.7,
#                          min_child_samples=21,
#                          min_child_weight=0.001,
#                          bagging_fraction = 1,
#                          bagging_freq = 2,
#                          reg_alpha = 0.001,
#                          reg_lambda = 8,
#                          cat_smooth = 0,
#                          num_iterations = 200,   
#                         )
# gsearch = GridSearchCV(gbm, param_grid=parameters, scoring='roc_auc', cv=3)
# gsearch.fit(X_0_train, Y_0_train)
# print('参数的最佳取值:{0}'.format(gsearch.best_params_))
# print('最佳模型得分:{0}'.format(gsearch.best_score_))
# print(gsearch.cv_results_['mean_test_score'])
# print(gsearch.cv_results_['params'])

In [182]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
train_x, test_x, train_y, test_y = train_test_split(X_train,Y_train, test_size=0.2, random_state=2018)
model = model(train_x,train_y,test_x,test_y)

#0.00852019
#0.00855984
#0.00855797

Training until validation scores don't improve for 50 rounds
[50]	training's mape: 0.00870695	valid_1's mape: 0.00871581
[100]	training's mape: 0.00858872	valid_1's mape: 0.00866069
[150]	training's mape: 0.00849799	valid_1's mape: 0.0086129
[200]	training's mape: 0.00844774	valid_1's mape: 0.00859282
[250]	training's mape: 0.00840345	valid_1's mape: 0.00857292
[300]	training's mape: 0.0083754	valid_1's mape: 0.00856796
[350]	training's mape: 0.00835148	valid_1's mape: 0.00856114
Early stopping, best iteration is:
[335]	training's mape: 0.00835923	valid_1's mape: 0.00855797


In [183]:
y_lightgbm_all = dict()
for prod in X_test.itemID.unique():
    train_predict = pd.DataFrame(scaler.inverse_transform(pd.DataFrame(model.predict(X_test[X_test.itemID == prod].drop(['itemID'], axis=1))))).rename(columns={0:'predicted_count'})
    y_lightgbm_all [prod] = int(train_predict["predicted_count"].sum().round())


In [185]:
# perfect result
print(f'Perfect Result: {evaluate_result(y, y):.2f}')

# baseline 1
print(f'Baseline 1: {evaluate_result(y, y_baseline1):.2f}')

# baseline 2
print(f'Baseline 2: {evaluate_result(y, y_baseline2):.2f}')

# random forest
print(f'LightGBM: {evaluate_result(y, y_lightgbm_all ):.2f}')

Perfect Result: 7895975.87
Baseline 1: -3727365.60
Baseline 2: -1672504.21
LightGBM: 59135.11


In [ ]:
#54783.05
#59135.11